<a href="https://colab.research.google.com/github/MarcoRuizS/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/MarcoRuiz_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nombre: Marco Ruiz    
Link: https://colab.research.google.com/drive/18b0bJtX8bBa9_-KM4j-hVhJW53oQx91F <br>
Actividad Guiada 3

**EL PROBLEMA DEL AGENTE VIAJERO <br> TSP. TSPLIB Juegos de datos para poner a prueba nuestros diseños.**


In [1]:
# !pip install requests #Liberia para hacer llamadas http(descargar páginas y ficheros)

import urllib.request

file = "swiss42.tsp"
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f2320568b38>)

In [2]:
!pip install tsplib95

In [0]:
import tsplib95
import random
import math

In [0]:
#Datos del problema
problem = tsplib95.load_problem(file)

Numero de nodos y de aristas (42, del 0 al 41)

In [0]:
# Nodos
Nodos = list(problem.get_nodes())
# Aristas
Aristas = list(problem.get_edges())

In [6]:
Nodos
# random.choice(list(set(Nodos)  - set([0])))
list(set(Nodos)) # lista a tupla

set(Nodos)-set([2]) #elimina el nodo 

problem.wfunc(0,41)

124

Fuerza Bruta complejidad:(n-1)!/2 ; n = 42 (No aplica)

Técnica: Busqueda Aleatoria complejidad:

In [7]:
#Funciones Generales
#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
    solucion = [0]
    for i in range(len(Nodos)-1):
        solucion = solucion + [random.choice(list(set(Nodos)  - set(solucion)))]
    return solucion 

# crear_solucion(Nodos)

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
    return problem.wfunc(a,b)

# distancia(0,2,problem)

#Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problem):
    distancia_total = 0
    for i in range(len(solucion)-1):
        distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
    return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

solucion = crear_solucion(Nodos)  # 0 al 41
distancia_total(solucion, problem)

4771

In [8]:
#Busqueda Aleatoria
def busqueda_aleatoria(problem,N):
  mejor_solucion  = crear_solucion(Nodos) #solucion aleatoria
  mejor_valor     = distancia_total(mejor_solucion, problem)

  for i in range(N):
    solucion  = crear_solucion(Nodos)
    valor     = distancia_total(solucion, problem)
#     print(valor,'^', mejor_valor, '*',i)
    if valor < mejor_valor:
      mejor_solucion = solucion
      mejor_valor = valor

  print("La Mejor Solucion es: ", mejor_valor,'\nRecorrido:',mejor_solucion)

print('Con 100 iteraciones')
busqueda_aleatoria(problem,100)
print('\nCon 1000 iteraciones:')
busqueda_aleatoria(problem,1000)
print('\nCon 10000 iteraciones:')
busqueda_aleatoria(problem,10000)
print('\nCon 100000 iteraciones:')
busqueda_aleatoria(problem,100000)

Con 100 iteraciones
La Mejor Solucion es:  4145 
Recorrido: [0, 3, 20, 4, 30, 33, 26, 41, 22, 14, 28, 40, 6, 17, 15, 32, 36, 37, 1, 8, 24, 29, 18, 16, 13, 7, 21, 38, 2, 10, 39, 23, 5, 11, 25, 12, 9, 27, 19, 35, 31, 34]

Con 1000 iteraciones:
La Mejor Solucion es:  3946 
Recorrido: [0, 13, 19, 11, 37, 35, 36, 17, 22, 18, 7, 15, 5, 8, 32, 39, 41, 23, 25, 26, 6, 21, 10, 1, 2, 30, 12, 40, 9, 20, 31, 29, 33, 14, 24, 34, 38, 28, 16, 4, 3, 27]

Con 10000 iteraciones:
La Mejor Solucion es:  3464 
Recorrido: [0, 7, 6, 1, 34, 21, 41, 24, 32, 39, 40, 13, 26, 16, 36, 3, 28, 22, 9, 12, 23, 25, 11, 10, 30, 18, 29, 31, 17, 35, 5, 14, 37, 19, 15, 2, 38, 20, 33, 8, 27, 4]

Con 100000 iteraciones:
La Mejor Solucion es:  3529 
Recorrido: [0, 28, 21, 39, 9, 22, 38, 30, 5, 3, 20, 1, 23, 25, 41, 12, 11, 27, 36, 18, 19, 6, 32, 17, 35, 37, 15, 31, 33, 4, 34, 29, 8, 10, 16, 14, 7, 13, 26, 40, 2, 24]


Las soluciones se aproximan conforme aumenta el número de iteraciones, se realizo pruebas desde 100 hasta 10000, la mejor solución encontrada aun no es la optima ( 1273 )

Busqueda Local: Generador de Vecindad    

La idea es mejorar la solucion encontrada

In [9]:
#Generador de Vencidad

def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) 
  #Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
 
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] 
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  print('Mejor Distancia:', mejor_distancia,'\nRecorrido:', mejor_solucion)
  return mejor_solucion
      
solucion = crear_solucion(Nodos)
mejor_solucion = genera_vecina(solucion)


Mejor Distancia: 4527 
Recorrido: [0, 32, 27, 36, 5, 23, 30, 11, 9, 26, 29, 37, 1, 7, 6, 19, 15, 33, 34, 39, 20, 18, 21, 10, 28, 2, 24, 8, 16, 35, 31, 38, 4, 13, 25, 41, 12, 3, 22, 17, 40, 14]


In [22]:
# Busqueda Local
def busqueda_local(problem, N):
  #problem = datos del problema
  #N = numero de iteraciones
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  solucion_referencia = crear_solucion(Nodos)
  
  for i in range(N):
    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)
    if distancia_vecina <= mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
        
    solucion_referencia = vecina

  print("La mejor solución encontrada es: " , mejor_distancia ,'\nRecorrido:', mejor_solucion)

busqueda_local(problem, 50)

Mejor Distancia: 4048 
Recorrido: [0, 19, 20, 3, 10, 11, 1, 14, 15, 6, 30, 39, 25, 2, 31, 7, 40, 23, 22, 26, 28, 27, 34, 24, 38, 29, 21, 41, 18, 32, 12, 8, 36, 16, 5, 9, 33, 35, 13, 4, 17, 37]
Mejor Distancia: 3822 
Recorrido: [0, 19, 20, 3, 10, 11, 1, 14, 15, 6, 30, 39, 25, 2, 31, 7, 40, 23, 22, 26, 28, 27, 9, 24, 38, 29, 21, 41, 18, 32, 12, 8, 36, 16, 5, 34, 33, 35, 13, 4, 17, 37]
Mejor Distancia: 3599 
Recorrido: [0, 19, 20, 3, 10, 11, 1, 14, 15, 6, 30, 39, 25, 2, 31, 7, 40, 23, 22, 32, 28, 27, 9, 24, 38, 29, 21, 41, 18, 26, 12, 8, 36, 16, 5, 34, 33, 35, 13, 4, 17, 37]
Mejor Distancia: 3386 
Recorrido: [0, 19, 20, 3, 10, 11, 1, 14, 15, 6, 30, 39, 25, 2, 31, 7, 40, 23, 22, 32, 28, 27, 9, 24, 38, 29, 21, 41, 18, 26, 12, 8, 13, 16, 5, 34, 33, 35, 36, 4, 17, 37]
Mejor Distancia: 3150 
Recorrido: [0, 19, 20, 3, 10, 11, 1, 14, 15, 6, 30, 39, 25, 2, 4, 7, 40, 23, 22, 32, 28, 27, 9, 24, 38, 29, 21, 41, 18, 26, 12, 8, 13, 16, 5, 34, 33, 35, 36, 31, 17, 37]
Mejor Distancia: 2942 
Recorrido: [

mediante Generador de vecindad y busqueda locar mejoramos el resultado anterior.   


RECOCIDO SIMULADO

In [11]:
#Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos)
def genera_vecina_aleatorio(solucion):
   #Se puede mejorar haciendo que la elección no se uniforme sino entre las que estén más proximas
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  if j<i:
    i,j=j,i
  vecina = list(solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:])
  return vecina

solucion = crear_solucion(Nodos)
print('Recorrido 1:',solucion,'\n\nRecorrido generado:', genera_vecina_aleatorio(solucion))

Recorrido 1: [0, 14, 21, 3, 2, 23, 20, 41, 6, 9, 37, 29, 36, 10, 1, 19, 11, 7, 15, 24, 22, 31, 33, 27, 25, 39, 8, 30, 34, 5, 4, 40, 18, 16, 32, 13, 26, 17, 38, 35, 28, 12] 

Recorrido generado: [0, 14, 21, 3, 2, 23, 20, 41, 6, 9, 37, 29, 36, 10, 1, 34, 11, 7, 15, 24, 22, 31, 33, 27, 25, 39, 8, 30, 19, 5, 4, 40, 18, 16, 32, 13, 26, 17, 38, 35, 28, 12]


In [0]:
#Funcion de probabilidad para determinar si se cambia 
#    a una solución peor respecto a la de referencia(exponencial)
def probabilidad(T,d):
  if random.random() <= math.exp(-1*d / T)   :
    return True
  else:
    return False

def bajar_temperatura(T):
  return T*.9999

In [14]:
#Recocido Simulado
#####################################################################
def recocido_simulado(problem, TEMPERATURA=1000):
  #problem = datos del problema

  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100

  while TEMPERATURA > 1: #criterio de parada
    #Genera una solución vecina(aleatoria)
    vecina = genera_vecina_aleatorio(solucion_referencia) 
    
    #Calcula su valor(distancia)
    distancia_vecina = distancia_total(vecina, problem)
    
    #Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      print('\t**Encontrada mejor solucion:', mejor_distancia , mejor_solucion, '\n')
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según 
    # una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    p = probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) )
    if distancia_vecina < distancia_referencia or p :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
    #bajo la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es :" , mejor_distancia,'\nRecorrido:', mejor_solucion  )
  return mejor_solucion

sol = recocido_simulado(problem, TEMPERATURA=1000)

	**Encontrada mejor solucion: 4857 [0, 19, 24, 34, 38, 2, 3, 40, 27, 4, 18, 20, 31, 15, 6, 11, 41, 5, 7, 32, 39, 30, 8, 17, 25, 37, 12, 14, 33, 22, 26, 29, 23, 13, 36, 28, 9, 10, 35, 16, 21, 1] 

	**Encontrada mejor solucion: 4831 [0, 19, 24, 34, 38, 2, 3, 40, 27, 4, 18, 20, 31, 15, 6, 11, 41, 5, 7, 16, 39, 30, 8, 17, 25, 37, 12, 14, 33, 22, 26, 29, 23, 13, 36, 28, 9, 10, 35, 32, 21, 1] 

	**Encontrada mejor solucion: 4799 [0, 19, 24, 34, 38, 2, 3, 40, 13, 4, 18, 20, 31, 15, 6, 11, 41, 5, 14, 16, 39, 30, 8, 17, 25, 37, 12, 7, 33, 22, 26, 29, 23, 27, 36, 28, 9, 10, 35, 32, 21, 1] 

	**Encontrada mejor solucion: 4570 [0, 19, 24, 34, 38, 2, 3, 40, 13, 4, 18, 20, 31, 15, 6, 11, 41, 1, 14, 16, 39, 30, 8, 17, 25, 37, 12, 7, 33, 22, 21, 29, 23, 27, 36, 28, 9, 10, 35, 32, 26, 5] 

	**Encontrada mejor solucion: 4564 [0, 19, 6, 30, 29, 40, 12, 2, 17, 7, 20, 18, 14, 15, 24, 11, 41, 1, 22, 16, 39, 34, 35, 13, 37, 25, 3, 4, 31, 33, 23, 38, 9, 27, 36, 28, 21, 10, 8, 32, 26, 5] 

	**Encontrada mejor 

**Colonia de Hormigas**

In [0]:
#Funciones
def Add_Nodo(problem, H ,T ) :
  #Establecer una una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )


def Incrementa_Feromona(problem, T, H):
  #Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T):
  #Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [0]:
def Add_Nodo_feromona(problem, H ,T):
    # funcion para añadir un nuevo nodo dependiendo de: los nodos mas cercanos, feromonas 
    Nodos = list(problem.get_nodes())
    opciones = list(set(range(1,len(Nodos)))-set(H)) 
    aux = [0 for i in range(len(opciones))] # almacenar feromonas
    probabilidad = [0 for i in range(len(opciones))] # almacena probabilidad
    denominador = 0

    #una vez almacenado recorro el vector para comparar cantida de feromonas
    #dejadas y la distacia del ultimo punto recorrido de la hormiga y el siguiente
    #posible punto (inv_dist) 
    
    for i in range(len(opciones)): #recorro todas las opciones
        inv_dist = 1000/distancia(H[-1], opciones[i], problem)
        cant_ferom_arista = T[H[-1]][opciones[i]]
        media = math.sqrt(((cant_ferom_arista)**2+(inv_dist)**2)/2) # mejor feromona, distancia
        aux[i] = media # guardo en el vector de feromonas
        denominador += media
    
    #inicializo las probabilidades, tomo la mayor
    mayor_nodo = 0
    maxim_prob = 0
    
    for i in range(len(opciones)):
        probabilidad[i] = aux[i]/denominador
        if probabilidad[i] > maxim_prob: #si es la maxima probalilidad lo guardo
            maxim_prob = probabilidad[i]
            mayor_nodo = i #guardo el nodo con mayor probabilidad
    #Devuelve el nodo con mayor probabilidad
    return opciones[mayor_nodo]

In [0]:
#Colonia de Hormigas

def hormigas(problem, N, feromona) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)
    
    # Nodos
  Nodos = list(problem.get_nodes())
    #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
#######################################
  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      if feromona:
         Nuevo_Nodo =Add_Nodo_feromona(problem, Hormiga[h] , T)
      else:
         Nuevo_Nodo =  Add_Nodo(problem, Hormiga[h] ,T )
            
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual
  
  print("\nLa mejor solución encontrada es: " , mejor_distancia ,'\nRecorrido:', mejor_solucion)


In [21]:
hormigas(problem, 100 ,False)
hormigas(problem, 100 ,True)


La mejor solución encontrada es:  4233 
Recorrido: [0, 11, 27, 1, 41, 30, 26, 31, 32, 13, 20, 37, 39, 21, 33, 35, 34, 36, 14, 12, 23, 9, 8, 22, 40, 2, 6, 24, 17, 29, 19, 15, 28, 38, 25, 18, 5, 7, 16, 3, 10, 4]

La mejor solución encontrada es:  1630 
Recorrido: [0, 1, 6, 4, 3, 2, 27, 28, 29, 30, 32, 34, 20, 33, 31, 17, 37, 15, 14, 16, 19, 13, 5, 26, 18, 12, 11, 25, 10, 8, 9, 23, 41, 21, 39, 22, 38, 24, 40, 7, 36, 35]


Para 100 hormigas, tomando la elección de nodos aleatorio, se obtuvo un resultado muy alto para una solución aceptable, no llegamos a valores muy buenos. Mediante la selección de nodos con las mejores distancias y feromonas, con 100 hormigas, se pudo conseguir un valor mucho mejor, no se obtuvo un valor optimo, pero es muy aceptable.